# Safety and Utilization of the Valencia Street Center Bike Lane

## Background

In the summer of 2023, the SFMTA launched an 18-month pilot program to test a center-running bike lane along Valencia Street. This redesign replaced the street’s traditional side-running bike lanes—located between parked cars and moving traffic—with a single, bidirectional bike lane positioned in the center of the road. Northbound and southbound bicycle traffic shared this lane, separated by a painted white line, while vehicle traffic ran on either side, divided from the bike lane by rubber curbs and flexible bollards.

The project aimed to improve safety for cyclists while preserving both parking and vehicle through-traffic. The center-running design was expected to eliminate several common hazards: cyclists being “doored” by people exiting parked cars, drivers making right turns across bike lanes, and vehicles merging out from curbside parking. The addition of physical barriers—bollards and rubber curbs—also offered a level of protection that the previous side-running lanes lacked entirely.

## Investigation Questions

This analysis seeks to evaluate the impact of the Valencia center-running bike lane by addressing the following questions:
- Did the center-running bike lane improve safety for cyclists?
- Did the center-running bike lane lead to an increase in the number of cyclists using Valencia Street?
- How did the types and severity of crashes under the center-running design compare to those under the previous side-running bike lanes?



In [ ]:
# import packages
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import logging
from IPython.display import HTML
import plotly.io as pio

pio.renderers.default = 'notebook'  
logging.basicConfig(level=logging.INFO)


## Data Description

### Data Sources

This analysis draws on multiple data sources to assess changes in bike safety and ridership during the Valencia center-running bike lane pilot.

* **Traffic Crash Data**
  Comprehensive data on traffic collisions was obtained from the [SF Gov “Traffic Crashes Resulting in Injury” dataset](https://data.sfgov.org/Public-Safety/Traffic-Crashes-Resulting-in-Injury/ubvf-ztfx/about_data). This dataset includes police-reported crashes citywide involving injuries to motorists, pedestrians, or cyclists. It provides details such as crash type, location, date/time, and parties involved—making it essential for identifying trends in bicycle-related crashes before, during, and after the pilot.

* **Bike Volume Data (SFMTA Counters)**
  Ridership trends were analyzed using the [SFMTA’s automated bike counter data](https://www.sfmta.com/reports/average-weekday-bike-volumes-dashboard). These counters track the number of bicycles passing fixed locations throughout the city, including Valencia Street.

* **Bay Wheels Trip Data**
  Additional insights into cyclist activity were drawn from the [Bay Wheels system data](https://www.lyft.com/bikes/bay-wheels/system-data), which provides anonymized trip-level records from the city’s bike-share program. This dataset includes start and end locations, timestamps, and bike types, helping capture short-term shifts in ridership that might not be reflected in fixed counters alone.

### Data Inclusion

THe traffic crash dataset only includes crashes reported by SFPD. There are likely more bicycle related crashes that went unreported so it should be assumed that this analysis overestimates actual bicycle safety. The dataset includes traffic crashes reported between January 1st, 2005 to May 31st, 2025. Crashes are restricted to San Francisco city limmits.

The complete baywheels dataset is too large to load into memory, so the data included is restricted to rides that either start or end on Valencia Street, with the assumption that these rides traveled via the center bike lane. Additionally only data from the following years is included,[2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025].

### Data Indexing and Valencia Center Bike Lane Definition 

Bicycle crash data was extracted from the full crash dataset by identifying records where a bicyclist was listed among the involved party types. Crashes specific to Valencia Street were indexed by matching the **primary road name** to “Valencia Street.” To isolate crashes occurring within the center-running bike lane corridor, spatial filtering was applied to include only those with **latitude coordinates between 15th and 23rd Streets**.

Although the center-running bike lane was in place from **August 1, 2023, to February 1, 2025**, this analysis only uses crash data from **calendar year 2024**. Data from **August to December 2023** was excluded to avoid potential bias introduced by road users adapting to the new street configuration. By **January 2024**, it is assumed that most users were familiar with the layout, making crash patterns more representative of typical conditions. Data from **2025** was also excluded due to temporary road obstructions and construction-related disruptions during the lane’s removal. To account for the limited time window, the analysis emphasizes **population-weighted and trend-based metrics**, rather than raw counts.


In [2]:
# import crash data
crash_data = pd.read_csv('../data/Traffic_Crashes_Resulting_in_Injury_20250714.csv')
# Convert collision_datetime to datetime with the desired format
crash_data['collision_datetime'] = pd.to_datetime(
    crash_data['collision_datetime'], 
    format='%m/%d/%Y %I:%M:%S %p', 
    errors='coerce'
)
start_date = crash_data['collision_datetime'].min()
end_date = crash_data['collision_datetime'].max()
print(f"Crash data date range: {start_date} to {end_date}")

# stringify crash party types
def get_crash_party_type(row):
    party_type1 = row['party1_type']
    party_type2 = row['party2_type']
    if pd.isna(party_type1) or pd.isna(party_type2):
        return 'unknown'
    if ('bic' in party_type1.lower() or 'bic' in party_type2.lower()) and ('driv' in party_type1.lower() or 'driv' in party_type2.lower()):
        return 'car_on_bike'
    elif ('bic' in party_type1.lower() and 'bic' in party_type2.lower()):
        return 'bike_on_bike'
    elif ('ped' in party_type1.lower() or 'ped' in party_type2.lower()) and ('driv' in party_type1.lower() or 'driv' in party_type2.lower()):
        return 'car_on_pedestrian'
    elif ('ped' in party_type1.lower() or 'ped' in party_type2.lower()) and ('bic' in party_type1.lower() or 'bic' in party_type2.lower()):
        return 'bike_on_pedestrian'
    elif ('driv' in party_type1.lower() and 'driv' in party_type2.lower()):
        return 'car_on_car'
    elif ('ped' in party_type1.lower() and 'ped' in party_type2.lower()):
        return 'pedestrian_on_pedestrian'
    else:
        return 'other'
    
def car_action(row):
    party1_type = row['party1_type']
    party2_type = row['party2_type']
    if pd.isna(party1_type) or pd.isna(party2_type):
        return None
    if 'driv' in party1_type.lower() or 'driv' in party2_type.lower():
        return row['party1_move_pre_acc'] if 'driv' in party1_type.lower() else row['party2_move_pre_acc']
    return None

def bike_action(row):
    party1_type = row['party1_type']
    party2_type = row['party2_type']
    if pd.isna(party1_type) or pd.isna(party2_type):
        return None
    if 'bic' in party1_type.lower() or 'bic' in party2_type.lower():
        return row['party1_move_pre_acc'] if 'bic' in party1_type.lower() else row['party2_move_pre_acc']
    return None

crash_data['crash_party_type'] = crash_data.apply(get_crash_party_type, axis=1)
crash_data['car_action'] = crash_data.apply(car_action, axis=1)
crash_data['bike_action'] = crash_data.apply(bike_action, axis=1)

/var/folders/ry/6zlsyrrj22zffwv1bszfkwz80000gn/T/ipykernel_29785/917009861.py:2: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  crash_data = pd.read_csv('../data/Traffic_Crashes_Resulting_in_Injury_20250714.csv')


Crash data date range: 2005-01-01 01:15:00 to 2025-05-31 22:01:00


In [3]:
# date range of crash data
crash_data['collision_datetime'] = pd.to_datetime(crash_data['collision_datetime'], errors='coerce')


In [4]:
# Load the bike count data
bike_counter_data = pd.read_csv('../data/BikeCountsTable.csv')

# Add a datetime column
bike_counter_data['datetime'] = pd.to_datetime(
    bike_counter_data['Month of Date'] + ' ' + bike_counter_data['Hour'].astype(str) + ':00:00',
    format='%B %Y %H:%M:%S',
    errors='coerce'
)

In [5]:

# load baywheels data
years_to_load = [2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
chunk_size = 100_000

all_pieces = []

# if '../data/processed_valencia_baywheels_data.csv' exists, load it
try:
    baywheels_data = pd.read_csv('../data/processed_valencia_baywheels_data.csv')
    logging.info("Loaded pre-processed baywheels data.")
    # visualize baywheels data

    # Convert 'started_at' to datetime
    # Try parsing with milliseconds first, then fallback to seconds-only format
    parsed = pd.to_datetime(
        baywheels_data['started_at'],
        format="%Y-%m-%d %H:%M:%S.%f",
        errors="coerce"
    )
    mask = parsed.isna()
    if mask.any():
        parsed.loc[mask] = pd.to_datetime(
            baywheels_data.loc[mask, 'started_at'],
            format="%Y-%m-%d %H:%M:%S",
            errors="coerce"
        )
    baywheels_data['started_at'] = parsed
except FileNotFoundError:
    logging.info("Pre-processed baywheels data not found, processing raw data...")

    for year in years_to_load:
        print(f"Loading baywheels data for {year}")
        file_path = f'../data/baywheels_{year}_combined.csv'

        try:
            # 1) Figure out which time column exists
            header = pd.read_csv(file_path, nrows=0)
            if 'started_at' in header.columns:
                time_col = 'started_at'
            elif 'start_time' in header.columns:
                time_col = 'start_time'
            else:
                raise ValueError(f"Neither 'started_at' nor 'start_time' found in {file_path}")

            usecols = ['start_station_name', 'end_station_name', time_col]

            # 2) Stream in chunks
            for chunk in pd.read_csv(file_path,
                                    usecols=usecols,
                                    chunksize=chunk_size):

                # 3) Rename to unified 'started_at'
                if time_col != 'started_at':
                    chunk = chunk.rename(columns={time_col: 'started_at'})

                # 4) Parse the timestamps in this chunk
                #    a) try fractional‐seconds format first
                parsed = pd.to_datetime(
                    chunk['started_at'],
                    format="%Y-%m-%d %H:%M:%S.%f",
                    errors="coerce"
                )
                #    b) fallback to plain‐seconds format where needed
                mask = parsed.isna()
                if mask.any():
                    parsed.loc[mask] = pd.to_datetime(
                        chunk.loc[mask, 'started_at'],
                        format="%Y-%m-%d %H:%M:%S",
                        errors="coerce"
                    )
                chunk['started_at'] = parsed

                # 5) Filter for Valencia
                mask_val = (
                    chunk['start_station_name']
                        .str
                        .contains("Valencia", case=False, na=False)
                    |
                    chunk['end_station_name']
                        .str
                        .contains("Valencia", case=False, na=False)
                )
                all_pieces.append(chunk.loc[mask_val])

        except FileNotFoundError:
            print(f"  ⚠️ File not found: {file_path}")
        except ValueError as ve:
            print(f"  ⚠️ {ve}")

    # 6) One final concat
    if all_pieces:
        baywheels_data = pd.concat(all_pieces, ignore_index=True)
    else:
        baywheels_data = pd.DataFrame(columns=[
            'start_station_name', 'end_station_name', 'started_at'
        ])

    # 7) (Optional) Double‐check dtype & NaTs
    print("started_at dtype:", baywheels_data['started_at'].dtype)
    print("unparsed timestamps:", baywheels_data['started_at'].isna().sum())

    # 8) Add year_month
    baywheels_data['year_month'] = baywheels_data['started_at'].dt.to_period('M')
    print(f"baywheels_data size: {baywheels_data.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")


INFO:root:Loaded pre-processed baywheels data.


In [6]:
# Replace your saving cell with this:
#baywheels_data['started_at'] = pd.to_datetime(baywheels_data['started_at'], errors='coerce')
#baywheels_data.to_csv('../data/processed_valencia_baywheels_data.csv', index=False)
#logging.info("Saved processed baywheels data to '../data/processed_valencia_baywheels_data.csv'.")

In [7]:

# index the data
# grab crashes that invovle bikes 
ii_bike_crashes =(
    (crash_data['party1_type'].str.contains('bic', case=False, na=False)) |
    (crash_data['party2_type'].str.contains('bic', case=False, na=False))
)   
# grab crahses that happen on valencia street 
ii_valencia = (
    (crash_data['primary_rd'].str.contains('VALENCIA ST', case=False, na=False)) 
)

# grab crashes that happen between 15th and 23rd on Valencia
latitude_range = (37.766433, 37.753915)
ii_center_bike_lane_geo = (
    (crash_data['tb_latitude'] < latitude_range[0]) &
    (crash_data['tb_latitude'] > latitude_range[1]) &
    ii_valencia
)

# grab crashes that occured during the center bike lane time period
start_date = pd.to_datetime('2023-08-01 00:00:00')
end_date = pd.to_datetime('2025-02-01 00:00:00')
ii_center_bike_lane_time = (
    (crash_data['collision_datetime'] >= start_date) &
    (crash_data['collision_datetime'] < end_date)
)


## Crash and Ride Count Data Analysis

### Comparing Bicycle Crashes: Valencia Street and Citywide Patterns

Before diving into the center bike lane impact, it's useful to contextualize bicycle crash metrics on Valencia street against the rest of San Francisco. Several visualizations are presented to explore crash types, crash severity, driver behavior preceeding a crash, party involvement, and time of day between bicycle crashes on Valencia and those citywide. 

Crashes on Valencia Street are similar to the rest of the city in terms of severity and road users implicated (although during the time span analyzed there have been no fatal bike-related accidents reported on Valencia). However, Valencia has a significantly higher fraction of "parked car" crashes -- indicative of a cyclicst getting "doored" -- as compared to the top categories of crashes prevelant city-wide. In terms of timing, Valencia also shows a disproportionately higher concentration of crashes during the evening rush hour (5–8 p.m.), with peak morning crashes representing only 30.6% of peak evening volumes. This contrasts with citywide trends, where morning crashes reach approximately 77.8% of evening peak levels.


In [8]:

# Filter Valencia bike crashes
valencia_bike_crashes = crash_data[ii_valencia & ii_bike_crashes].copy()

# Metrics for Valencia bike crashes
valencia_crash_types = valencia_bike_crashes['type_of_collision'].value_counts()
valencia_crash_severity = valencia_bike_crashes['collision_severity'].value_counts()
valencia_crash_times = valencia_bike_crashes['collision_datetime'].dt.hour.value_counts().sort_index()

# Metrics for all bike crashes
all_bike_crash_types = crash_data[ii_bike_crashes]['type_of_collision'].value_counts()
all_bike_crash_severity = crash_data[ii_bike_crashes]['collision_severity'].value_counts()
all_bike_crash_times = crash_data[ii_bike_crashes]['collision_datetime'].dt.hour.value_counts().sort_index()

# Create subplots for crash types
fig_types_pie = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                              subplot_titles=["Types of Bike Crashes<br>on Valencia Street", "Types of Bike Crashes<br>(All Streets)"])

# Valencia Street pie chart
fig_types_pie.add_trace(
    go.Pie(labels=valencia_crash_types.index, values=valencia_crash_types.values, name="Valencia Street"),
    row=1, col=1
)

# All Streets pie chart
fig_types_pie.add_trace(
    go.Pie(labels=all_bike_crash_types.index, values=all_bike_crash_types.values, name="All Streets"),
    row=1, col=2
)

# Update layout
fig_types_pie.update_layout(title_text="Comparison of Crash Types: Valencia Street vs All Streets")
fig_types_pie.show()


# Add car action analysis
# Create subplots for car action comparison
fig_car_action = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                               subplot_titles=["Top 5 Car Actions<br>Valencia Street", "Top 5 Car Actions<br>(All Streets)"])

# Get car actions for Valencia Street (top 5)
valencia_car_action = valencia_bike_crashes.groupby(['car_action']).size()
valencia_car_action = valencia_car_action.dropna()
valencia_car_action_sorted = valencia_car_action.sort_values(ascending=False)
top_5_valencia_actions = valencia_car_action_sorted.head(5)

# Get car actions for All Streets (top 5)
all_bike_crashes = crash_data[ii_bike_crashes]
all_car_action = all_bike_crashes.groupby(['car_action']).size()
all_car_action = all_car_action.dropna()
all_car_action_sorted = all_car_action.sort_values(ascending=False)
top_5_all_actions = all_car_action_sorted.head(5)

# Valencia Street car actions pie chart
fig_car_action.add_trace(
    go.Pie(
        labels=top_5_valencia_actions.index,
        values=top_5_valencia_actions.values,
        name="Valencia Street"
    ),
    row=1, col=1
)

# All Streets car actions pie chart
fig_car_action.add_trace(
    go.Pie(
        labels=top_5_all_actions.index,
        values=top_5_all_actions.values,
        name="All Streets"
    ),
    row=1, col=2
)

# Update layout
fig_car_action.update_layout(title_text="Comparison of Top 5 Car Actions: Valencia Street vs All Streets")
fig_car_action.show()

# Create subplots for crash severity
fig_severity_pie = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                                 subplot_titles=["Severity of Bike Crashes<br>on Valencia Street", "Severity of Bike Crashes<br>(All Streets)"])

# Valencia Street pie chart
fig_severity_pie.add_trace(
    go.Pie(labels=valencia_crash_severity.index, values=valencia_crash_severity.values, name="Valencia Street"),
    row=1, col=1
)

# All Streets pie chart
fig_severity_pie.add_trace(
    go.Pie(labels=all_bike_crash_severity.index, values=all_bike_crash_severity.values, name="All Streets"),
    row=1, col=2
)

# Update layout
fig_severity_pie.update_layout(title_text="Comparison of Crash Severity: Valencia Street vs All Streets")
fig_severity_pie.show()

# Create a figure for time of crashes
fig_times_combined = go.Figure()

# Add Valencia Street data
fig_times_combined.add_trace(
    go.Scatter(
        x=valencia_crash_times.index,
        y=valencia_crash_times.values,
        mode='lines+markers',
        name='Valencia Street',
        line=dict(color='blue')
    )
)

# Add All Streets data
fig_times_combined.add_trace(
    go.Scatter(
        x=all_bike_crash_times.index,
        y=all_bike_crash_times.values,
        mode='lines+markers',
        name='All Streets',
        line=dict(color='red'),
        yaxis='y2'  # Assign to secondary y-axis
    )
)

# look at crash party type
# Create subplots for crash party type comparison
fig_party_pie = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                              subplot_titles=["Crash Party Types<br>on Valencia Street", "Crash Party Types<br>(All Streets)"])

# Get crash party types for Valencia and All Streets
valencia_party_types = valencia_bike_crashes['crash_party_type'].value_counts()
all_party_types = crash_data[ii_bike_crashes]['crash_party_type'].value_counts()

# Format labels by replacing underscores with spaces and capitalizing
valencia_party_labels = [label.replace('_', ' ').title() for label in valencia_party_types.index]
all_party_labels = [label.replace('_', ' ').title() for label in all_party_types.index]

# Valencia Street pie chart
fig_party_pie.add_trace(
    go.Pie(labels=valencia_party_labels, values=valencia_party_types.values, name="Valencia Street"),
    row=1, col=1
)

# All Streets pie chart
fig_party_pie.add_trace(
    go.Pie(labels=all_party_labels, values=all_party_types.values, name="All Streets"),
    row=1, col=2
)

# Update layout
fig_party_pie.update_layout(title_text="Comparison of Crash Party Types: Valencia Street vs All Streets")
fig_party_pie.show()

# Update layout for dual y-axes and move the legend
fig_times_combined.update_layout(
    title="Time of Bike Crashes: Valencia Street vs All Streets",
    xaxis_title="Hour of Day",
    yaxis=dict(
        title="Valencia Street Crash Count",
        showgrid=True, # Enable grid lines for the left y-axis
        range=(0, 50),
        nticks=10,  # Set number of ticks for the left y-axis
    ),
    yaxis2=dict(
        title="All Streets Crash Count",
        overlaying='y',  # Overlay the right y-axis on the left y-axis
        side='right',
        showgrid=False,  # Enable grid lines for the right y-axis
        range=(0, 1000),
        nticks=10  # Set number of ticks for the right y-axis
    ),
    legend=dict(
        title="Crash Location",
        x=0.01,  # Move legend to the top left
        y=0.99   # Move legend vertically near the top
    )
)

# Show the plot
fig_times_combined.show()

# show percentage of morning peak crashes vs evening peak crashes

print(f"Valencia Morning Peak vs Evening Peak: {valencia_crash_times[7:10].max() / valencia_crash_times[16:19].max() * 100:.2f}%")
print(f"All bike crashes Morning Peak vs Evening Peak: {all_bike_crash_times[7:10].max() / all_bike_crash_times[16:19].max() * 100:.2f}%")
# grab the center lane crash where the car was parked

Valencia Morning Peak vs Evening Peak: 30.61%
All bike crashes Morning Peak vs Evening Peak: 77.81%


### Spatial and Temporal Distribution of Valencia Bike Crashes

Analyzing the spatial and temporal distribution of bicycle crashes on Valencia Street helps place individual months and locations in the context of broader patterns. This perspective is useful for identifying trends as well as flagging potential outliers or anomalies.

Over the sampled time period, the average number of crashes per month on Valencia Street was **1.44**, with a variance of **1.14**. The maximum number of crashes observed in a single month was **six**, which occurred only twice—an event with an estimated probability of just **0.82%**, suggesting these were statistically rare occurrences.

Crash locations along Valencia Street were not evenly distributed. The blocks near **16th, 17th, and 18th Streets** recorded the highest individual crash counts. Within this central segment of the corridor, crash locations were relatively balanced between **intersections and midblock segments**. However, certain areas stand out as having distinct patterns. For example, **Brosnan Street** stands out: **92%** of crashes there occurred within **20 feet** of the intersection, indicating a localized risk pattern.


In [15]:
# where do valencia bike crashes happen?

# Get the cross streets from valencia bike crashes with intersection data
cross_street_intersection_data = valencia_bike_crashes.groupby(['secondary_rd', 'intersection']).size().unstack(fill_value=0)

# Remove any rows where secondary_rd is NaN and get top 15 cross streets by total crashes
cross_street_intersection_data = cross_street_intersection_data.dropna()
cross_street_totals = cross_street_intersection_data.sum(axis=1).sort_values(ascending=False)
top_15_streets = cross_street_totals.head(15).index
cross_street_intersection_data = cross_street_intersection_data.loc[top_15_streets]

# Create a stacked bar chart
fig_cross_streets = go.Figure()

# Define colors for each intersection type (crimson-orange palette)
colors = {
    'Midblock > 20ft': '#FFB347',           # vibrant orange-red
    'Intersection <= 20ft': '#FF5733',      # orange
    'Intersection Rear End <= 150ft': '#FF8C42'  # light orange
}

# Add each intersection type as a separate bar
for intersection_type in cross_street_intersection_data.columns:
    fig_cross_streets.add_trace(go.Bar(
        name=intersection_type,
        x=cross_street_intersection_data.index,
        y=cross_street_intersection_data[intersection_type],
        marker_color=colors.get(intersection_type, 'gray')
    ))

# Update layout for stacked bar chart
fig_cross_streets.update_layout(
    title='Valencia Street Bike Crashes by Cross Street and Location Type (Top 15)',
    xaxis_title='Cross Street',
    yaxis_title='Number of Crashes',
    xaxis_tickangle=-45,
    barmode='stack',
    legend_title='Crash Location Type'
)

fig_cross_streets.show()

# Extract year-month from valencia_bike_crashes
valencia_bike_crashes_copy = valencia_bike_crashes.copy()
valencia_bike_crashes_copy['year_month'] = valencia_bike_crashes_copy['collision_datetime'].dt.to_period('M')

# Count crashes per month
crashes_per_month_raw = valencia_bike_crashes_copy['year_month'].value_counts().sort_index()

# Create a complete date range from min to max date in the entire crash dataset
all_crash_dates = crash_data['collision_datetime'].dropna()
date_range = pd.period_range(
    start=all_crash_dates.min().to_period('M'),
    end=all_crash_dates.max().to_period('M'),
    freq='M'
)

# Create a complete dataframe with all months, filling missing values with 0
complete_df = pd.DataFrame({'year_month': date_range})
crashes_per_month_df = pd.DataFrame({
    'year_month': crashes_per_month_raw.index,
    'crash_count': crashes_per_month_raw.values
})

# Merge to get complete time series with zeros for missing months
complete_crashes = complete_df.merge(crashes_per_month_df, on='year_month', how='left')
complete_crashes['crash_count'] = complete_crashes['crash_count'].fillna(0)

# Convert back to Series for consistency with existing code
crashes_per_month = pd.Series(
    data=complete_crashes['crash_count'].values,
    index=complete_crashes['year_month']
).sort_index()

# Create histogram
fig_histogram = go.Figure()

fig_histogram.add_trace(go.Histogram(
    x=crashes_per_month.values,
    nbinsx=20,
    histnorm='probability',  # This normalizes to show probability
    name='Valencia Bike Crashes',
    marker_color='blue',
    opacity=0.6
))

fig_histogram.update_layout(
    title='Distribution of Valencia Street Bike Crash Frequency',
    xaxis_title='Number of Crashes per Month',
    yaxis_title='Probability',
    showlegend=False
)

fig_histogram.show()

# Print summary statistics
print(f"Valencia bike crashes summary statistics:")
print(f"Total months with data: {len(crashes_per_month)}")
print(f"Mean crashes per month: {crashes_per_month.mean():.2f}")
print(f"Median crashes per month: {crashes_per_month.median():.2f}")
print(f"Standard deviation: {crashes_per_month.std():.2f}")


# Print time range information
print(f"Time period covered: {crashes_per_month.index.min()} to {crashes_per_month.index.max()}")
print(f"Total time span: {len(crashes_per_month)} months")

Valencia bike crashes summary statistics:
Total months with data: 245
Mean crashes per month: 1.44
Median crashes per month: 1.00
Standard deviation: 1.29
Time period covered: 2005-01 to 2025-05
Total time span: 245 months


### Comparing Types and Severity of Crashes on The Valencia Street Center Bike Lane vs. Side-Running

The crash data on Valencia Street is divided into two periods: crashes that occurred during the center-running bike lane pilot in 2024, and crashes that occurred both before and after the pilot. To ensure consistency, both datasets are limited to the same geographic corridor between 15th and 23rd Streets. Absolute crash metrics are reported for each period. Notably, only 13 crashes were recorded during the center bike lane pilot, compared to 187 crashes outside the pilot period. As such, it’s important to recognize that the center lane data represents a much smaller sample size and is therefore more vulnerable to statistical noise.

This analysis addresses the investigation question: *How did the types and severity of crashes under the center-running design compare to those under the previous side-running bike lanes?* As expected, the center bike lane in 2024 had zero crashes resulting from vehicles making right turns into cyclists, and zero crashes from parked cars -- which infers no cyclists were "doored". Both of these crash mechanisms were highly prevelant with side running bike lanes. There was a relative increase in bike-on-bike and bike-on-pedestrian crashes during the center lane period, though this shift should be interpreted with caution due to the small sample size. Overall, crash severity remained broadly similar between the two periods.

In terms of location, Sycamore Street had the highest number of crashes during the 2024 pilot, followed by 16th Street and the segment spanning 17th to 20th Streets. The elevated crash count near 16th Street is consistent with pre-pilot patterns. However, the spike in crashes at Sycamore Street stands out as a potential anomaly: in 2024 alone, there were four crashes at or near Sycamore, compared to only seven crashes at that location over the previous 17.5 years.


In [18]:
# index the data for center bike lane crashes
center_lane_crashes = crash_data[ii_valencia & ii_center_bike_lane_geo & ii_center_bike_lane_time & ii_bike_crashes]
center_lane_crashes_2024 = center_lane_crashes[center_lane_crashes['collision_datetime'].dt.year == 2024]
valencia_crashes = crash_data[ii_valencia & ii_bike_crashes & ii_center_bike_lane_geo & ~ii_center_bike_lane_time]

#valencia_crashes_2019 = valencia_crashes[valencia_crashes['collision_datetime'].dt.year == 2019]
#valencia_crashes = valencia_crashes_2019

#FIXME: make it clear the geo bounds on these plots

# Metrics for center lane crashes in 2024
center_crash_types = center_lane_crashes_2024['type_of_collision'].value_counts()
center_crash_severity = center_lane_crashes_2024['collision_severity'].value_counts()

# Metrics for Valencia crashes (before center bike lane)
valencia_crash_types = valencia_crashes['type_of_collision'].value_counts()
valencia_crash_severity = valencia_crashes['collision_severity'].value_counts()

# Create subplots for crash types comparison
fig_types_pie = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                              subplot_titles=["Types of Bike Crashes<br>Center Lane 2024", "Types of Bike Crashes<br>Valencia Before/After Center Lane"])

# Center lane crashes 2024 pie chart
fig_types_pie.add_trace(
    go.Pie(labels=center_crash_types.index, values=center_crash_types.values, name="Center Lane 2024"),
    row=1, col=1
)

# Valencia crashes (before center lane) pie chart
fig_types_pie.add_trace(
    go.Pie(labels=valencia_crash_types.index, values=valencia_crash_types.values, name="Before/After Center Lane"),
    row=1, col=2
)

# Update layout
fig_types_pie.update_layout(title_text="Comparison of Crash Types: Center Lane 2024 vs Before/After Center Lane")
fig_types_pie.show()


# Create subplots for car action comparison

# Get the cross streets from center lane crashes 2024 with intersection data
center_car_action = center_lane_crashes_2024.groupby(['car_action']).size()
center_car_action = center_car_action.dropna()
center_car_action_sorted = center_car_action.sort_values(ascending=False)
top_10_center_actions = center_car_action_sorted.head(5).index
center_car_action = center_car_action.loc[top_10_center_actions]

# Get the car actions from valencia crashes (before center lane)
valencia_car_action = valencia_crashes.groupby(['car_action']).size()
valencia_car_action = valencia_car_action.dropna()
valencia_car_action_sorted = valencia_car_action.sort_values(ascending=False)
top_10_valencia_actions = valencia_car_action_sorted.head(5).index
valencia_car_action = valencia_car_action.loc[top_10_valencia_actions]

fig_car_action = make_subplots(
    rows=1, cols=2, 
    specs=[[{'type': 'domain'}, {'type': 'domain'}]],
    subplot_titles=["Top 5 Car Actions<br>Center Lane 2024", "Top 5 Car Actions<br>Before/After Center Lane"]
)

# Center lane 2024 car actions pie chart
fig_car_action.add_trace(
    go.Pie(
        labels=center_car_action.index,
        values=center_car_action.values,
        name="Center Lane 2024"
    ),
    row=1, col=1
)

# Before center lane car actions pie chart
fig_car_action.add_trace(
    go.Pie(
        labels=valencia_car_action.index,
        values=valencia_car_action.values,
        name="Before Center Lane"
    ),
    row=1, col=2
)

# Update layout for stacked bar charts
fig_car_action.update_layout(
    title_text="Comparison of Car Actions by Intersection Type: Center Lane 2024 vs Before/After Center Lane",
    height=600,
    barmode='stack',
    legend_title='Car Action Type'
)

# Update x-axis labels for both subplots
fig_car_action.update_xaxes(tickangle=-45, title_text="Car Action")
fig_car_action.update_yaxes(title_text="Number of Car Actions")

fig_car_action.show()

# look at crash party type
# Create subplots for crash party type comparison
fig_party_pie = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                              subplot_titles=["Crash Party Types<br>on Center Lane 2024", "Crash Party Types<br>(Before/After Center Lane)"])

# Get crash party types for Center Lane 2024 and All Streets
center_lane_party_types = center_lane_crashes_2024['crash_party_type'].value_counts()
valencia_party_types = valencia_crashes['crash_party_type'].value_counts()

# Format labels by replacing underscores with spaces and capitalizing
center_lane_party_labels = [label.replace('_', ' ').title() for label in center_lane_party_types.index]
valencia_party_labels = [label.replace('_', ' ').title() for label in valencia_party_types.index]

# Center Lane 2024 pie chart
fig_party_pie.add_trace(
    go.Pie(labels=center_lane_party_labels, values=center_lane_party_types.values, name="Center Lane 2024"),
    row=1, col=1
)

# Before Center Lane pie chart
fig_party_pie.add_trace(
    go.Pie(labels=valencia_party_labels, values=valencia_party_types.values, name="Before/After Center Lane"),
    row=1, col=2
)

# Update layout
fig_party_pie.update_layout(title_text="Comparison of Crash Party Types: Center Lane 2024 vs Before/After Center Lane")
fig_party_pie.show()


# Create subplots for crash severity comparison
fig_severity_pie = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                                 subplot_titles=["Severity of Bike Crashes<br>Center Lane 2024", "Severity of Bike Crashes<br>Valencia Before/After Center Lane"])

# Center lane crashes 2024 pie chart
fig_severity_pie.add_trace(
    go.Pie(labels=center_crash_severity.index, values=center_crash_severity.values, name="Center Lane 2024"),
    row=1, col=1
)

# Valencia crashes (before center lane) pie chart
fig_severity_pie.add_trace(
    go.Pie(labels=valencia_crash_severity.index, values=valencia_crash_severity.values, name="Before/After Center Lane"),
    row=1, col=2
)

# Update layout
fig_severity_pie.update_layout(title_text="Comparison of Crash Severity: Center Lane 2024 vs Before/After Center Lane")
fig_severity_pie.show()

# Get the cross streets from center lane crashes 2024 with intersection data
center_cross_street_intersection_data = center_lane_crashes_2024.groupby(['secondary_rd', 'intersection']).size().unstack(fill_value=0)
center_cross_street_intersection_data = center_cross_street_intersection_data.dropna()
center_cross_street_totals = center_cross_street_intersection_data.sum(axis=1).sort_values(ascending=False)
top_10_center_streets = center_cross_street_totals.head(10).index
center_cross_street_intersection_data = center_cross_street_intersection_data.loc[top_10_center_streets]

# Get the cross streets from valencia crashes (before center lane) with intersection data
valencia_cross_street_intersection_data = valencia_crashes.groupby(['secondary_rd', 'intersection']).size().unstack(fill_value=0)
valencia_cross_street_intersection_data = valencia_cross_street_intersection_data.dropna()
valencia_cross_street_totals = valencia_cross_street_intersection_data.sum(axis=1).sort_values(ascending=False)
top_10_valencia_streets = valencia_cross_street_totals.head(10).index
valencia_cross_street_intersection_data = valencia_cross_street_intersection_data.loc[top_10_valencia_streets]

# Create subplots for cross streets comparison with stacked bars
fig_cross_streets = make_subplots(
    rows=1, cols=2, 
    specs=[[{'type': 'xy'}, {'type': 'xy'}]],
    subplot_titles=["Cross Streets by Location Type<br>Center Lane 2024", "Cross Streets by Location Type<br>Before/After Center Lane"]
)

# Define colors for each intersection type (same as the main stacked chart)
colors = {
    'Midblock > 20ft': '#FFB347',           # vibrant orange-red
    'Intersection <= 20ft': '#FF5733',      # orange
    'Intersection Rear End <= 150ft': '#FF8C42'  # light orange
}

# Add center lane 2024 stacked bars
for intersection_type in center_cross_street_intersection_data.columns:
    fig_cross_streets.add_trace(
        go.Bar(
            name=intersection_type,
            x=center_cross_street_intersection_data.index,
            y=center_cross_street_intersection_data[intersection_type],
            marker_color=colors.get(intersection_type, 'gray'),
            showlegend=False
        ),
        row=1, col=1
    )

# Add before center lane stacked bars
for intersection_type in valencia_cross_street_intersection_data.columns:
    fig_cross_streets.add_trace(
        go.Bar(
            name=intersection_type,
            x=valencia_cross_street_intersection_data.index,
            y=valencia_cross_street_intersection_data[intersection_type],
            marker_color=colors.get(intersection_type, 'gray'),
            showlegend=True
        ),
        row=1, col=2
    )

# Update layout for stacked bar charts
fig_cross_streets.update_layout(
    title_text="Comparison of Crash Locations by Intersection Type: Center Lane 2024 vs Before/After Center Lane",
    height=600,
    barmode='stack',
    legend_title='Crash Location Type'
)

# Update x-axis labels for both subplots
fig_cross_streets.update_xaxes(tickangle=-45, title_text="Cross Street")
fig_cross_streets.update_yaxes(title_text="Number of Crashes")

fig_cross_streets.show()

# Print summary statistics for comparison
print(f"Center lane crashes in 2024: {len(center_lane_crashes_2024)}")
print(f"Valencia crashes before center lane: {len(valencia_crashes)}")

Center lane crashes in 2024: 13
Valencia crashes before center lane: 187


In [11]:
# grab the center lane crash that occured on "SYCAMORE ST"

sycamore_crash = center_lane_crashes_2024[center_lane_crashes_2024['secondary_rd'] == 'SYCAMORE ST'].reset_index(drop=True)

print(f"2024 Sycamore Street crash details:\n{sycamore_crash[['collision_datetime', 'type_of_collision', 'collision_severity', 'car_action','bike_action']].head()}")

2024 Sycamore Street crash details:
   collision_datetime type_of_collision          collision_severity  \
0 2024-09-16 08:30:00         Sideswipe  Injury (Complaint of Pain)   
1 2024-03-17 22:23:00           Head-On             Injury (Severe)   
2 2024-06-28 19:25:00         Broadside      Injury (Other Visible)   
3 2024-09-04 18:00:00         Broadside  Injury (Complaint of Pain)   

            car_action          bike_action  
0     Making Left Turn  Proceeding Straight  
1  Proceeding Straight           Not Stated  
2        Making U Turn  Proceeding Straight  
3     Making Left Turn  Proceeding Straight  


### Quantifying Bicycle Traffic Patterns on Valencia Street

To evaluate the safety impact of the center-running bike lane, it’s useful to measure crashes relative to the number of bike trips—essentially, crashes per ride. This metric and accompanying analysis will aim to answer the investigation questions, *did the center-running bike lane improve safety for cyclists?* and, *did the center-running bike lane lead to an increase in the number of cyclists using Valencia Street?*

Answering these questions requires reliable estimates of how many cyclists used Valencia during the pilot. The SFMTA operates several automated bike counters across the city, including two on Valencia Street. However, these sensors can be [unreliable](https://www.sfmta.com/reports/average-weekday-bike-volumes-dashboard) due to downtime and incomplete coverage. To supplement this, Bay Wheels trip data—specifically rides starting or ending on Valencia—was used as a proxy for broader cycling activity.

BaBike counter data shows only a slight increase in traffic past the Duboce–14th Street counter compared with the previous two years. In contrast, Bay Wheels trips to and from Valencia rose sharply during the center-running lane period: in 2024, rides increased by 25.6% compared with 2022. While this suggests greater cycling activity, the Bay Wheels dataset has limitations based on the assumptions stated previously, and furthermore, the increase could partly reflect growing use of the bike-share service itself rather than the new infrastructure. This analysis does not attempt to separate those effects.


In [12]:
# look at bike counter data
valencia_locations = ['Valencia St. SB Between Duboce & 14th',
                      'Valencia St. Totem NB Between 16th & 17th']

bike_counter_structure = {}
for location in valencia_locations:
    # Filter bike counter data for each location
    filtered_data = bike_counter_data[bike_counter_data['Counter Location'].str.contains(location, case=False, na=False)]
    
    # Add to the structure
    bike_counter_structure[location] = filtered_data


In [13]:
# Extract year and month for grouping and plot both locations on the same axes
fig_rides_month = go.Figure()

colors = {
    'Valencia St. SB Between Duboce & 14th': 'blue',
    'Valencia St. Totem NB Between 16th & 17th': 'orange'
}

for key, valencia_counter_data_orig in bike_counter_structure.items():

    # Create a copy to avoid the warning
    valencia_counter_data = valencia_counter_data_orig.copy()
    
    # Convert 'Month of Date' to datetime
    valencia_counter_data['Month of Date'] = pd.to_datetime(valencia_counter_data['Month of Date'], format='%B %Y', errors='coerce')
    
    # Extract year and month
    valencia_counter_data['year_month'] = valencia_counter_data['Month of Date'].dt.to_period('M')
    
    # Group by year and month
    rides_by_month = valencia_counter_data.groupby('year_month')['Total Bike Count'].sum().reset_index(name='ride_count')
    
    # Create a complete date range from min to max date
    date_range = pd.period_range(
        start=rides_by_month['year_month'].min(),
        end=rides_by_month['year_month'].max(),
        freq='M'
    )
    
    # Create a complete dataframe with all months
    complete_df = pd.DataFrame({'year_month': date_range})
    
    # Merge with actual data, leaving missing values as NaN
    rides_by_month = complete_df.merge(rides_by_month, on='year_month', how='left')
    
    # Convert 'year_month' to string for Plotly compatibility
    rides_by_month['year_month'] = rides_by_month['year_month'].astype(str)

    # get yearly totals
    yearly_totals = valencia_counter_data.copy()
    yearly_totals['year'] = yearly_totals['year_month'].dt.year
    yearly_totals = yearly_totals.groupby('year')['Total Bike Count'].sum().reset_index(name='yearly_total')

    
    # Add each location's trace to the same figure
    fig_rides_month.add_trace(
        go.Scatter(
            x=rides_by_month['year_month'],
            y=rides_by_month['ride_count'],
            mode='lines+markers',
            name=key.replace("_", " ").title(),
            connectgaps=False,  # This will break the line at NaN values
            line=dict(width=2, color=colors.get(key, 'gray'))
        )
    )

    # Add yearly bar chart (hidden by default)
    fig_rides_month.add_trace(
        go.Bar(
            x=yearly_totals['year'],
            y=yearly_totals['yearly_total'],
            name='Yearly Total ' + key.replace("_", " ").title(),
            marker_color=colors.get(key, 'gray'),
            opacity=0.6,
            text=yearly_totals['yearly_total'],
            textposition='auto',
            visible=False  # Hidden by default
        )
    )


# Add toggle buttons
fig_rides_month.update_layout(
    title='Rides vs Month/Year (Valencia Street Counters)',
    xaxis_title='Time Period',
    yaxis_title='Ride Count',
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"visible": [True, False]},
                          {"title": "Bike Counter Data: Monthly Trends",
                           "xaxis.title": "Month/Year",
                           "yaxis.title": "Monthly Ride Count"}],
                    label="Monthly View",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, True]},
                          {"title": "Bike Counter Data: Yearly Totals",
                           "xaxis.title": "Year",
                           "yaxis.title": "Yearly Total Rides"}],
                    label="Yearly View",
                    method="update"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.01,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ],
    legend=dict(
        x=0.65,
        y=0.99
    ),
    height=500
)

    

fig_rides_month.update_layout(
    title='Rides vs Month/Year (Valencia Street Counters)',
    xaxis_title='Month/Year',
    yaxis_title='Ride Count'
)
fig_rides_month.show()




In [19]:

# Extract year and month for grouping
baywheels_data['year_month'] = baywheels_data['started_at'].dt.to_period('M')

# Group by year and month
rides_by_month = baywheels_data.groupby('year_month').size().reset_index(name='ride_count')

# Convert 'year_month' to period[M] for merging
rides_by_month['year_month'] = rides_by_month['year_month'].astype('period[M]')

# Create a complete date range from min to max date
date_range = pd.period_range(
    start=rides_by_month['year_month'].min(),
    end=rides_by_month['year_month'].max(),
    freq='M'
)

# Create a complete dataframe with all months
complete_df = pd.DataFrame({'year_month': date_range})

# Merge with actual data, leaving missing values as NaN
rides_by_month = complete_df.merge(rides_by_month, on='year_month', how='left')

# Convert 'year_month' to string for Plotly compatibility
rides_by_month['year_month'] = rides_by_month['year_month'].astype(str)

# Calculate yearly totals for Bay Wheels
baywheels_data['year'] = baywheels_data['started_at'].dt.year
yearly_totals = baywheels_data.groupby('year').size().reset_index(name='yearly_total')

# Plot rides vs month/year with toggle functionality
fig_rides_month = go.Figure()

# Add monthly line chart (visible by default)
fig_rides_month.add_trace(
    go.Scatter(
        x=rides_by_month['year_month'],
        y=rides_by_month['ride_count'],
        mode='lines+markers',
        name='Monthly (Bay Wheels)',
        connectgaps=False,
        line=dict(color='green', width=2),
        marker=dict(size=6),
        visible=True  # Visible by default
    )
)

# Add yearly bar chart (hidden by default)
fig_rides_month.add_trace(
    go.Bar(
        x=yearly_totals['year'],
        y=yearly_totals['yearly_total'],
        name='Yearly Total (Bay Wheels)',
        marker_color='green',
        opacity=0.6,
        text=yearly_totals['yearly_total'],
        textposition='auto',
        visible=False  # Hidden by default
    )
)

# Add toggle buttons
fig_rides_month.update_layout(
    title='Bay Wheels Valencia Street Rides: Monthly vs Yearly View',
    xaxis_title='Time Period',
    yaxis_title='Ride Count',
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"visible": [True, False]},
                          {"title": "Bay Wheels Valencia Street Rides: Monthly Trends",
                           "xaxis.title": "Month/Year",
                           "yaxis.title": "Monthly Ride Count"}],
                    label="Monthly View",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, True]},
                          {"title": "Bay Wheels Valencia Street Rides: Yearly Totals",
                           "xaxis.title": "Year",
                           "yaxis.title": "Yearly Total Rides"}],
                    label="Yearly View",
                    method="update"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.01,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ],
    legend=dict(
        x=0.01,
        y=0.99
    ),
    height=500
)

fig_rides_month.show()


In [20]:
# Add analysis of bike count vs hour of the day for bike counter data
fig_hourly_counter = go.Figure()

for key, valencia_counter_data_orig in bike_counter_structure.items():
    # Create a copy to avoid the warning
    valencia_counter_data = valencia_counter_data_orig.copy()
    
    # Group by hour and sum the bike counts
    hourly_counts = valencia_counter_data.groupby('Hour')['Total Bike Count'].sum().reset_index()
    
    # Add trace for each location
    fig_hourly_counter.add_trace(
        go.Scatter(
            x=hourly_counts['Hour'],
            y=hourly_counts['Total Bike Count'],
            mode='lines+markers',
            name=key.replace("_", " ").title(),
            line=dict(width=2),
            marker=dict(size=6)
        )
    )

# Add analysis of Bay Wheels rides by hour of day
baywheels_data['hour'] = baywheels_data['started_at'].dt.hour

# Group by hour and count rides
hourly_baywheels = baywheels_data.groupby('hour').size().reset_index(name='ride_count')

fig_hourly_counter.add_trace(
    go.Scatter(
        x=hourly_baywheels['hour'],
        y=hourly_baywheels['ride_count'],
        mode='lines+markers',
        name='Bay Wheels Valencia',
        line=dict(color='green', width=2),
        marker=dict(size=6)
    )
)

fig_hourly_counter.update_layout(
    title='Valencia Street Bike Counts by Hour of Day (SFMTA Counters)',
    xaxis_title='Hour of Day',
    yaxis_title='Total Bike Count',
    xaxis=dict(range=[0, 23], dtick=2),
    height=500
)
fig_hourly_counter.show()



### Quantifying Bicycle Safety on Valencia Street as a Crash-per-Ride Metric

BayWheels trip counts and SFMTA bike counter data were used separately to calculate a “crashes-per-10,000-rides” metric. While the absolute values are subject to the limitations and assumptions of each counting method, the relative change in this metric before vs. during the center bike lane can help infer the lane’s impact on bicycle safety. Low crash counts make the analysis sensitive to noise, which limits the statistical strength of monthly fluctuations. For this reason, the analysis focuses on two perspectives: (1) peak monthly crash-per-ride values and (2) yearly averages, the latter carrying greater weight because they draw from a larger sample size. The comparison spans 2018, 2019, 2022, and 2024.

During the center bike lane period, peak crash-per-ride rates occurred in September and October 2024, reaching roughly 1.5 and 2 crashes per 10,000 rides for the SFMTA bike counter and BayWheels datasets, respectively. These rates exceeded that of peak month in 2022 (0.86 and 1.5, both in November) but are comparable to peaks observed in 2018 and 2019.

When averaged across the pre-center-lane years (2018, 2019, and 2022), the crash-per-ride rate was 0.5415 crashes per 10,000 rides past the 14th Street counter and 0.8302 crashes per 10,000 BayWheels trips. In 2024, those averages dropped to 0.4476 and 0.5964, respectively. While the center lane dataset covers only one year, the yearly averages—being based on far more rides than any single month—provide a stronger, less noise-driven indicator of change. On this basis, the data suggests a meaningful improvement in bicycle safety from a per-ride perspective.


In [21]:
years_to_analyze = [2018, 2019, 2022, 2024]

valencia_crashes = crash_data[ii_valencia & ii_bike_crashes & ii_center_bike_lane_geo]

# Create year_month for valencia crashes
valencia_crashes_filtered = valencia_crashes[valencia_crashes['collision_datetime'].dt.year.isin(years_to_analyze)]
valencia_crashes_filtered['year_month'] = valencia_crashes_filtered['collision_datetime'].dt.to_period('M')

# Count crashes per month
crashes_per_month = valencia_crashes_filtered.groupby('year_month').size().reset_index(name='crash_count')

# Create the line plot
fig_crashes_per_ride = go.Figure()

colors = {
    'Valencia St. SB Between Duboce & 14th': 'blue',
    'Valencia St. Totem NB Between 16th & 17th': 'orange',
    'Bay Wheels Valencia': 'green'
}

# Process each Valencia location separately
for key, valencia_counter_data_orig in bike_counter_structure.items():
    valencia_counter_data = valencia_counter_data_orig.copy()
    
    # Convert 'Month of Date' to datetime and create year_month
    valencia_counter_data['Month of Date'] = pd.to_datetime(valencia_counter_data['Month of Date'], format='%B %Y', errors='coerce')
    valencia_counter_data['year_month'] = valencia_counter_data['Month of Date'].dt.to_period('M')
    
    # Filter for years to analyze
    valencia_counter_data_filtered = valencia_counter_data[valencia_counter_data['year_month'].dt.year.isin(years_to_analyze)]
    
    # Group by year_month and sum bike counts for this location
    monthly_counts = valencia_counter_data_filtered.groupby('year_month')['Total Bike Count'].sum().reset_index(name='total_bike_count')

    # Merge crashes and bike counts for this location
    merged_data = pd.merge(crashes_per_month, monthly_counts, on='year_month', how='outer')
    
    # Fill missing values
    merged_data['crash_count'] = merged_data['crash_count'].fillna(0)
    merged_data = merged_data.dropna(subset=['total_bike_count'])  # Drop months with no bike count data

    # Calculate crashes per ride count (multiply by 10000 to get crashes per 10,000 rides for better readability)
    merged_data['crashes_per_10000_rides'] = (merged_data['crash_count'] / merged_data['total_bike_count']) * 10000
    
    # Sort by year_month
    merged_data = merged_data.sort_values('year_month')
    
    # Convert year_month to string for plotting
    merged_data['year_month_str'] = merged_data['year_month'].astype(str)
    
    # Clean up location name for legend
    location_name = key.replace('Valencia St. ', '').replace('Between ', '').replace(' & ', '-')
    
    # Group data by year to create separate traces for each year group
    year_groups = merged_data.groupby(merged_data['year_month'].dt.year)
    
    for i, (year, year_data) in enumerate(year_groups):
        # Only show legend for the first trace of each location
        show_legend = i == 0
        
        fig_crashes_per_ride.add_trace(
            go.Scatter(
                x=year_data['year_month_str'],
                y=year_data['crashes_per_10000_rides'],
                mode='lines+markers',
                name=f'{location_name}',
                line=dict(width=3),
                marker=dict(size=8),
                showlegend=show_legend,
                legendgroup=location_name,  # Group all traces for this location
                connectgaps=False,
                line_color=colors.get(key, 'gray')
            )
        )
    
# Process Bay Wheels data
baywheels_filtered = baywheels_data[baywheels_data['year_month'].dt.year.isin(years_to_analyze)]

# Group by year_month and count Bay Wheels rides
baywheels_monthly = baywheels_filtered.groupby('year_month').size().reset_index(name='baywheels_count')

# Merge crashes and Bay Wheels counts
baywheels_merged = pd.merge(crashes_per_month, baywheels_monthly, on='year_month', how='outer')

# Fill missing values
baywheels_merged['crash_count'] = baywheels_merged['crash_count'].fillna(0)
baywheels_merged = baywheels_merged.dropna(subset=['baywheels_count'])  # Drop months with no Bay Wheels data

# Calculate crashes per Bay Wheels ride
baywheels_merged['crashes_per_10000_rides'] = (baywheels_merged['crash_count'] / baywheels_merged['baywheels_count']) * 10000

# Sort by year_month
baywheels_merged = baywheels_merged.sort_values('year_month')

# Convert year_month to string for plotting
baywheels_merged['year_month_str'] = baywheels_merged['year_month'].astype(str)

# Group Bay Wheels data by year to create separate traces
baywheels_year_groups = baywheels_merged.groupby(baywheels_merged['year_month'].dt.year)

for i, (year, year_data) in enumerate(baywheels_year_groups):
    # Only show legend for the first trace
    show_legend = i == 0
    
    fig_crashes_per_ride.add_trace(
        go.Scatter(
            x=year_data['year_month_str'],
            y=year_data['crashes_per_10000_rides'],
            mode='lines+markers',
            name='Bay Wheels Valencia',
            line=dict(width=3),
            marker=dict(size=8),
            showlegend=show_legend,
            legendgroup='Bay Wheels Valencia',
            connectgaps=False,
            line_color=colors['Bay Wheels Valencia']
        )
    )

# Add center bike lane implementation period as shaded region
fig_crashes_per_ride.add_vrect(
    x0='2023-08',
    x1='2025-02',
    fillcolor='rgba(255, 0, 0, 0.2)',
    layer='below',
    line_width=0,
    annotation_text="Center Bike Lane Period",
    annotation_position="top left"
)

fig_crashes_per_ride.update_layout(
    title='Valencia Street Bike Crashes per 10,000 Rides by Month<br>(SFMTA Counters & Bay Wheels Data)',
    xaxis_title='Month/Year',
    yaxis_title='Crashes per 10,000 Rides',
    xaxis_tickangle=-45,
    height=500,
    showlegend=True
)

fig_crashes_per_ride.show()


/var/folders/ry/6zlsyrrj22zffwv1bszfkwz80000gn/T/ipykernel_29785/3121197010.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:

# Initialize data collection for the summary table
summary_data = []

valencia_locations = 'Valencia St. SB Between Duboce & 14th'

valencia_counter_data = bike_counter_structure[valencia_locations].copy()
# Convert 'Month of Date' to datetime and create year_month
valencia_counter_data['Month of Date'] = pd.to_datetime(valencia_counter_data['Month of Date'], format='%B %Y', errors='coerce')
valencia_counter_data['year_month'] = valencia_counter_data['Month of Date'].dt.to_period('M')
valencia_counter_data['year'] = valencia_counter_data['year_month'].dt.year

# Filter for years to analyze
valencia_counter_data_filtered = valencia_counter_data[valencia_counter_data['year'].isin(years_to_analyze)]

# Group by year and sum bike counts
annual_counts = valencia_counter_data_filtered.groupby('year')['Total Bike Count'].sum().reset_index()

# Get crash counts by year
valencia_crashes_annual = valencia_crashes_filtered.copy()
valencia_crashes_annual['year'] = valencia_crashes_annual['collision_datetime'].dt.year
annual_crashes = valencia_crashes_annual.groupby('year').size().reset_index(name='crash_count')

# Merge and calculate crashes per ride
annual_merged = pd.merge(annual_crashes, annual_counts, on='year', how='outer')
annual_merged['crash_count'] = annual_merged['crash_count'].fillna(0)
annual_merged = annual_merged.dropna(subset=['Total Bike Count'])
annual_merged['crashes_per_10000_rides'] = (annual_merged['crash_count'] / annual_merged['Total Bike Count']) * 10000

# Clean up location name
location_name = valencia_locations.replace('Valencia St. ', '').replace('Between ', '').replace(' & ', '-')

# Add to summary data
for _, row in annual_merged.iterrows():
    summary_data.append({
        'Data Source': location_name,
        'Year': int(row['year']),
        'Total Rides': f"{int(row['Total Bike Count']):,}",
        'Total Crashes': int(row['crash_count']),
        'Crashes per 10,000 Rides': f"{row['crashes_per_10000_rides']:.3f}"
    })

# Add Valencia St. Totem NB Between 16th & 17th data
totem_location = 'Valencia St. Totem NB Between 16th & 17th'
if totem_location in bike_counter_structure:
    totem_counter_data = bike_counter_structure[totem_location].copy()
    # Convert 'Month of Date' to datetime and create year_month
    totem_counter_data['Month of Date'] = pd.to_datetime(totem_counter_data['Month of Date'], format='%B %Y', errors='coerce')
    totem_counter_data['year_month'] = totem_counter_data['Month of Date'].dt.to_period('M')
    totem_counter_data['year'] = totem_counter_data['year_month'].dt.year
    
    # Filter for 2018, 2019, 2022 only (exclude 2024)
    totem_years = [2018, 2019, 2022]
    totem_counter_data_filtered = totem_counter_data[totem_counter_data['year'].isin(totem_years)]
    
    # Group by year and sum bike counts
    totem_annual_counts = totem_counter_data_filtered.groupby('year')['Total Bike Count'].sum().reset_index()
    
    # Filter crashes for the same years
    totem_crashes = valencia_crashes_filtered[valencia_crashes_filtered['collision_datetime'].dt.year.isin(totem_years)]
    totem_annual_crashes = totem_crashes.groupby(totem_crashes['collision_datetime'].dt.year).size().reset_index(name='crash_count')
    totem_annual_crashes.columns = ['year', 'crash_count']
    
    # Merge and calculate crashes per ride
    totem_annual_merged = pd.merge(totem_annual_crashes, totem_annual_counts, on='year', how='outer')
    totem_annual_merged['crash_count'] = totem_annual_merged['crash_count'].fillna(0)
    totem_annual_merged = totem_annual_merged.dropna(subset=['Total Bike Count'])
    totem_annual_merged['crashes_per_10000_rides'] = (totem_annual_merged['crash_count'] / totem_annual_merged['Total Bike Count']) * 10000
    
    # Clean up location name
    totem_location_name = totem_location.replace('Valencia St. ', '').replace('Between ', '').replace(' & ', '-')
    
    # Add to summary data for 2018, 2019, 2022
    for _, row in totem_annual_merged.iterrows():
        summary_data.append({
            'Data Source': totem_location_name,
            'Year': int(row['year']),
            'Total Rides': f"{int(row['Total Bike Count']):,}",
            'Total Crashes': int(row['crash_count']),
            'Crashes per 10,000 Rides': f"{row['crashes_per_10000_rides']:.3f}"
        })
    
    # Add N/A entry for 2024
    summary_data.append({
        'Data Source': totem_location_name,
        'Year': 2024,
        'Total Rides': 'N/A',
        'Total Crashes': 'N/A',
        'Crashes per 10,000 Rides': 'N/A'
    })

# Process Bay Wheels data for annual summaries
baywheels_annual = baywheels_filtered.copy()
baywheels_annual['year'] = baywheels_annual['year_month'].dt.year
baywheels_annual_counts = baywheels_annual.groupby('year').size().reset_index(name='baywheels_count')

# Get crash counts by year (same as above)
annual_crashes = valencia_crashes_filtered.groupby(valencia_crashes_filtered['collision_datetime'].dt.year).size().reset_index(name='crash_count')
annual_crashes.columns = ['year', 'crash_count']

# Merge Bay Wheels data
baywheels_annual_merged = pd.merge(annual_crashes, baywheels_annual_counts, on='year', how='outer')
baywheels_annual_merged['crash_count'] = baywheels_annual_merged['crash_count'].fillna(0)
baywheels_annual_merged = baywheels_annual_merged.dropna(subset=['baywheels_count'])
baywheels_annual_merged['crashes_per_10000_rides'] = (baywheels_annual_merged['crash_count'] / baywheels_annual_merged['baywheels_count']) * 10000

# Add Bay Wheels data to summary
for _, row in baywheels_annual_merged.iterrows():
    summary_data.append({
        'Data Source': 'Bay Wheels Valencia',
        'Year': int(row['year']),
        'Total Rides': f"{int(row['baywheels_count']):,}",
        'Total Crashes': int(row['crash_count']),
        'Crashes per 10,000 Rides': f"{row['crashes_per_10000_rides']:.3f}"
    })

# Create DataFrame and sort
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values(['Data Source', 'Year'])

# Create HTML table with styling
html_table = f"""
<div style="margin: 20px 0;">
<h3 style="color: #2c3e50; margin-bottom: 15px;">Valencia Street Bike Safety Summary ({', '.join(map(str, years_to_analyze))})</h3>
<table style="border-collapse: collapse; width: 100%; max-width: 800px; margin: 0 auto; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
  <thead>
    <tr style="background-color: #3498db; color: white;">
      <th style="padding: 12px; text-align: center; border: 1px solid #ddd;">Ride Count Source</th>
      <th style="padding: 12px; text-align: center; border: 1px solid #ddd;">Year</th>
      <th style="padding: 12px; text-align: center; border: 1px solid #ddd;">Total Rides</th>
      <th style="padding: 12px; text-align: center; border: 1px solid #ddd;">Total Crashes</th>
      <th style="padding: 12px; text-align: center; border: 1px solid #ddd;">Crashes per 10,000 Rides</th>
    </tr>
  </thead>
  <tbody>
"""

# Add rows with alternating colors
for i, (_, row) in enumerate(summary_df.iterrows()):
    bg_color = "#f8f9fa" if i % 2 == 0 else "#ffffff"
    
    # Highlight 2024 data (center bike lane period)
    if row['Year'] == 2024:
        bg_color = "#fff3cd"  # Light yellow highlight
    
    html_table += f"""
    <tr style="background-color: {bg_color};">
      <td style="padding: 10px; border: 1px solid #ddd; text-align: center; font-weight: 500;">{row['Data Source']}</td>
      <td style="padding: 10px; border: 1px solid #ddd; text-align: center;">{row['Year']}</td>
      <td style="padding: 10px; border: 1px solid #ddd; text-align: center; font-family: monospace;">{row['Total Rides']}</td>
      <td style="padding: 10px; border: 1px solid #ddd; text-align: center;">{row['Total Crashes']}</td>
      <td style="padding: 10px; border: 1px solid #ddd; text-align: center; font-weight: bold; color: #d63384;">{row['Crashes per 10,000 Rides']}</td>
    </tr>
    """

html_table += """
  </tbody>
</table>
<div style="margin-top: 15px; font-size: 0.9em; color: #6c757d; text-align: center;">
  <p><strong>Note:</strong> 2024 data highlighted in yellow represents the center bike lane period. Crash data covers the same geographic area (15th-23rd St) for all years. N/A indicates data not available for that period.</p>
</div>
</div>
"""

# Display the HTML table
display(HTML(html_table))


Ride Count Source,Year,Total Rides,Total Crashes,"Crashes per 10,000 Rides"
Bay Wheels Valencia,2018,"123,781",15,1.212
Bay Wheels Valencia,2019,"148,366",13,0.876
Bay Wheels Valencia,2022,"173,532",9,0.519
Bay Wheels Valencia,2024,"217,915",13,0.597
SB Duboce-14th,2018,"200,850",15,0.747
SB Duboce-14th,2019,"194,515",13,0.668
SB Duboce-14th,2022,"287,862",9,0.313
SB Duboce-14th,2024,"292,295",13,0.445
Totem NB 16th-17th,2018,"496,113",15,0.302
Totem NB 16th-17th,2019,"478,822",13,0.271


## Conclusion

### Ansering Investigation Questions

***Did the center-running bike lane improve safety for cyclists?***<br>***How did the types and severity of crashes under the center-running design compare to those under the previous side-running bike lanes?***

On a per-ride basis (accounting for the limitations of ride-count data), the center-running bike lane reduced the number of crashes—indicating a measurable improvement in cyclist safety. This reduction was largely driven by the elimination of certain high-risk scenarios: in 2024, there were no recorded crashes involving right turns into cyclists or “dooring” incidents. However, when crashes did occur, their severity mirrored that seen with the previous side-running bike lanes, meaning that the design did not improve injury outcomes.

***Did the center-running bike lane lead to an increase in the number of cyclists using Valencia Street?***

SFMTA’s bike counter just north of the lane’s starting point showed a slight increase in rides, while Bay Wheels data indicated a significant uptick in trips to and from Valencia. However, given the uncertainties and limitations of these counting methods, the data cannot definitively confirm an increase in overall bike lane usage.

### Additional Takeaways

One notable finding was the high frequency of crashes near Sycamore Street during the center-running lane’s operation. This location was a point where vehicles could cross the lane uninhibited—either by making U-turns or turning left from Sycamore onto Valencia. Of the four crashes recorded here, three could potentially have been prevented if the bike lane protection were extended through the intersection. Considering that only 13 crashes occurred in 2024, eliminating those three incidents would have significantly improved the safety metrics.

### Future Work

Valencia is a vibrant street for all road users, and the safety of pedestrians should also be considered when proposing and analyzing new road infrastructure projects. In a follow-on to this analysis, the impact of the center bike lane on pedestrain safety, as well as pedestrian safety on Valencia street in general will be analyzed. 

<br><br>
Author: Carlos Sama<br>Notebook and datasets: https://github.com/los-sama/valencia_bike_lane


